# Полиномиальная регрессия

## Стандартный базис

In [1]:
import numpy as np;
from sklearn.preprocessing import PolynomialFeatures
x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False) #подготовка базісных функций
poly.fit_transform(x[:, None]) #матрица значений М

array([[ 2.,  4.,  8.],
       [ 3.,  9., 27.],
       [ 4., 16., 64.]])

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
poly_model = make_pipeline(PolynomialFeatures(8), LinearRegression()) # модель полиномиальной регрессии для заданных базисных функци
#from sklearn.linear_model import Ridge
#poly_model = make_pipeline(PolynomialFeatures(30),Ridge(alpha=0.1))

In [3]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

In [4]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

poly_model.fit(x[:, np.newaxis], y) # обучение на заданных данных

xfit = np.linspace(0, 10, 1000)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y, c ='red')
plt.plot(xfit, yfit);

## полиномиальная регрессия самостоятельно

In [5]:
import numpy as np

class polyreg:
    
    def __init__(self,n):
        self.N = n
    
    def fit(self,x,y):
        F = np.zeros((len(x),self.N+1))
        
        for i in range(len(x)):
            for j in range(self.N+1):
                F[i][j] = x[i]**j # формируем матрицу значений М
        F_plus = np.matmul(np.linalg.inv(np.matmul(F.T,F)),F.T) # применяем формулу которую мы вывели на лекции
       # print(F_plus.shape)
        self.w = np.matmul(F_plus,y) # применяем формулу которую мы вывели на лекции для нахождения коэффициентов модели
        
    def predict(self, xtest):
        eval_mat = np.zeros((self.N+1,len(xtest)))
        
        for i in range(self.N+1):
            for j in range(len(xtest)):
                eval_mat[i][j] = xtest[j]**i 
       # print(eval_mat)
       # print(self.w)
       # print(self.w.dot(eval_mat))
        return self.w.dot(eval_mat)

In [6]:
import matplotlib.pyplot as plt

rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

model_1 = polyreg(8)
model_1.fit(x,y)

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
model_2 = make_pipeline(PolynomialFeatures(8), LinearRegression())

model_2.fit(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit_1 = model_1.predict(xfit[:, np.newaxis])
yfit_2 = model_2.predict(xfit[:, np.newaxis])



plt.scatter(x, y, c ='green')
plt.plot(xfit, yfit_1, 'r',linewidth = 2);

plt.plot(xfit, yfit_2, 'w--');

In [7]:
y

array([-0.92530881,  0.71111718, -0.06598087,  0.11672496,  0.88294471,
        0.8210899 ,  1.12370616, -0.23467501, -0.75446517, -0.86898322,
       -0.94231439,  0.70804351,  0.89495535,  0.53638242,  0.28955648,
        0.61914583, -0.84603144, -0.5796531 ,  1.01611705,  0.88180869,
        0.87399567, -0.28992469, -0.01353862,  0.65589053,  0.69771523,
        0.55374595,  0.78013085,  0.46920917,  0.91644209,  0.72516826,
        0.8837173 , -0.90676173, -0.10465615, -0.82186313,  0.70681199,
        0.13841844,  0.76810625,  0.74161023,  0.03745364,  0.88805266,
       -0.43137564,  1.01910093,  0.36236496,  0.7970268 ,  0.82783992,
       -0.89007576,  0.35538665,  0.28020998,  0.23855606,  0.94355877])

# Прогнозирование велотрафика (пример подготовки данных)

In [8]:
import pandas as pd
counts = pd.read_csv('fremont-bridge.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv('BicycleWeather.csv', index_col='DATE', parse_dates=True) #импортируем данные в объекты DataFrame

In [9]:
counts.head() # почасовые данные о велотрафике на мосту.

,West,East
Date,,
2012-10-03 00:00:00,4.0,9.0
2012-10-03 01:00:00,4.0,6.0
2012-10-03 02:00:00,1.0,1.0
2012-10-03 03:00:00,2.0,3.0
2012-10-03 04:00:00,6.0,1.0


In [10]:
weather.head()

,STATION,STATION_NAME,PRCP,SNWD,SNOW,TMAX,TMIN,AWND,WDF2,WDF5,...,WT17,WT05,WT02,WT22,WT04,WT13,WT16,WT08,WT18,WT03
DATE,,,,,,,,,,,,,,,,,,,,,
2012-01-01,GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,0,0,0,128,50,47,100,90,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
2012-01-02,GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,109,0,0,106,28,45,180,200,...,-9999,-9999,-9999,-9999,-9999,1,1,-9999,-9999,-9999
2012-01-03,GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,8,0,0,117,72,23,180,170,...,-9999,-9999,-9999,-9999,-9999,-9999,1,-9999,-9999,-9999
2012-01-04,GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,203,0,0,122,56,47,180,190,...,-9999,-9999,-9999,-9999,-9999,1,1,-9999,-9999,-9999
2012-01-05,GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,13,0,0,89,28,61,200,220,...,-9999,-9999,-9999,-9999,-9999,-9999,1,-9999,-9999,-9999


In [11]:
daily = counts.resample('D').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # преобразуем данные поучая суммарный траффик в течении суток.

In [12]:
daily.head()

,Total
Date,
2012-10-03,3521.0
2012-10-04,3475.0
2012-10-05,3148.0
2012-10-06,2006.0
2012-10-07,2142.0


In [13]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float) # вводім новый признак. день недели.

In [14]:
daily.head()

,Total,Mon,Tue,Wed,Thu,Fri,Sat,Sun
Date,,,,,,,,
2012-10-03,3521.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2012-10-04,3475.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2012-10-05,3148.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2012-10-06,2006.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2012-10-07,2142.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
daily['holiday'].fillna(0, inplace=True) # добавляем данные о праздниках

In [16]:
daily.head()

,Total,Mon,Tue,Wed,Thu,Fri,Sat,Sun,holiday
Date,,,,,,,,,
2012-10-03,3521.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-10-04,3475.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2012-10-05,3148.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2012-10-06,2006.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2012-10-07,2142.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
import numpy as np

def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))  # добавляем данные о длине светового дня
daily[['daylight_hrs']].plot()
plt.ylim(8, 17)

(8.0, 17.0)

In [18]:
weather['TMIN'] /= 10
weather['TMAX'] /= 10
weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# преобразования температуры. 
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)

daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']]) # преобразования температуры. 
# соединяем данные из двух файлов в одну переменную

In [19]:
daily['annual'] = (daily.index - daily.index[0]).days / 365.

In [20]:
daily.head()

,Total,Mon,Tue,Wed,Thu,Fri,Sat,Sun,holiday,daylight_hrs,PRCP,Temp (C),dry day,annual
Date,,,,,,,,,,,,,,
2012-10-03,3521.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.277359,0.0,13.35,1.0,0.000000
2012-10-04,3475.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.219142,0.0,13.60,1.0,0.002740
2012-10-05,3148.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.161038,0.0,15.30,1.0,0.005479
2012-10-06,2006.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.103056,0.0,15.85,1.0,0.008219
2012-10-07,2142.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.045208,0.0,15.85,1.0,0.010959


In [21]:
model.score(X,y)

NameError: name 'model' is not defined